In this workbook, I have built a topic moelling model using LDA 


The dataset used is of a micro lender company. 
The topic modelling is performed on the description of the loan application

In [10]:
!pip install unidecode
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [25]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7MB 5.5MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.2.2-py2.py3-none-any.whl size=135593 sha256=0d10083621ec485ca63068179c2811a59c66def176139a2566e6c30bd5f438e6
  Stored in directory: /root/.cache/pip/wheels/74/df/b6/97234c8446a43be05c9a8687ee0db1f1b5ade5f27729187eae
Successfully built pyLDAvis


In [12]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
kiva_df=pd.read_excel("Kiva_loans.xlsx")
kiva_df.info()

kiva_df = kiva_df.dropna()
kiva_df = kiva_df.reset_index(drop=True)

kiva_df.info()
kiva_df.shape
kiva_df.head()
kiva_df.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6818 entries, 0 to 6817
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   loan_id      6818 non-null   int64 
 1   status       6818 non-null   object
 2   sector       6818 non-null   object
 3   country      6818 non-null   object
 4   gender       6818 non-null   object
 5   loan_amount  6818 non-null   int64 
 6   nonpayment   6818 non-null   object
 7   en_clean     6802 non-null   object
dtypes: int64(2), object(6)
memory usage: 426.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6802 entries, 0 to 6801
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   loan_id      6802 non-null   int64 
 1   status       6802 non-null   object
 2   sector       6802 non-null   object
 3   country      6802 non-null   object
 4   gender       6802 non-null   object
 5   loan_amount  6802 non-null   int64

(6802, 8)

,loan_id,status,sector,country,gender,loan_amount,nonpayment,en_clean
0,0,defaulted,Agriculture,Kenya,M,500,lender,"Robert, 40, is married and has 6 children. In ..."
1,1,defaulted,Food,Kenya,F,500,lender,"Petronilla, 30, was deserted by her husband an..."
2,2,defaulted,Food,Kenya,M,500,lender,"Tom Mung'ahu, 45, is married and has 6 childre..."
3,3,defaulted,Services,Kenya,F,500,lender,"Benedina, 42, is married and has 4 girls. In a..."
4,4,defaulted,Construction,Kenya,M,500,lender,"Vincent Ondego 40, is married and has 8 childr..."


,loan_id,status,sector,country,gender,loan_amount,nonpayment,en_clean
6797,7983,paid,Clothing,Ecuador,F,1050,lender,Rosa is a member of the 3 de Mayo Communal Ban...
6798,7984,paid,Agriculture,Ecuador,F,2650,partner,"The communal bank, ""Cumbe,"" is located on the ..."
6799,7985,paid,Food,Ecuador,F,775,partner,Gladys Burgos Macas is a member of the 3 de Ma...
6800,7986,paid,Retail,Ecuador,F,275,partner,The Communal Bank Las Caleras is located in th...
6801,7987,paid,Services,Kenya,M,275,partner,Eliud Ouma owns the Purple Hut salon located o...


In [13]:
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
import re
import unidecode

#import warnings

#with warnings.catch_warnings():
#    warnings.filterwarnings("ignore",category=DeprecationWarning)

stop_words = set(stopwords.words('english') + stopwords.words('spanish'))

lemmer = WordNetLemmatizer()

def preprocess(x):
    # Lower case
    x = x.lower()
    
    # Remove punctuation
    x = re.sub(r'[^\w\s]', '', x)
    
    # Remove non-unicode
    x = unidecode.unidecode(x)
    
    # Remove numbers
    x = re.sub(r'\d+', '', x)
    
    # Remove stopwords and lemmatize
    x = [lemmer.lemmatize(w) for w in x.split() if w not in stop_words]
    return ' '.join(x) 

kiva_df['en_clean_pre'] = kiva_df['en_clean'].apply(preprocess)

In [14]:
kiva_df.iloc[0,:].en_clean

'Robert, 40, is married and has 6 children. In addition to his family of 8, he takes care of his mother and 5 brothers. Robert started by planting vegetables and selling at the local market. He then diversified with a tea nursery which is more profitable. Given a loan of $500, Robert will be able to improve his activities by buying fertilizers, pesticides, a pump and seedlings. The pump will remove the uncertainties of the weather. The current capital in his venture is Ksh 10500 or $150, which does not allow Robert to maximize his potential. Robert completed high school and has never got employment. He got apprentice training from a tea extension officer. He is a go getter, and his main hobby is teaching music.'

In [15]:
kiva_df.iloc[0,:].en_clean_pre

'robert married child addition family take care mother brother robert started planting vegetable selling local market diversified tea nursery profitable given loan robert able improve activity buying fertilizer pesticide pump seedling pump remove uncertainty weather current capital venture ksh allow robert maximize potential robert completed high school never got employment got apprentice training tea extension officer go getter main hobby teaching music'

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.5, min_df=0.05, 
                             max_features=1000, ngram_range=[1,3])
dtm = vectorizer.fit_transform(kiva_df['en_clean_pre'])
print(dtm.shape)


(6802, 270)


In [17]:
vectorizer.get_feature_names()

['able',
 'active',
 'active member',
 'activity',
 'add',
 'additional',
 'age',
 'ago',
 'allow',
 'also',
 'always',
 'amount',
 'another',
 'applied',
 'applied loan',
 'area',
 'attend',
 'back',
 'basic',
 'bean',
 'began',
 'belief',
 'better',
 'born',
 'bought',
 'boy',
 'business selling',
 'buy',
 'buying',
 'capital',
 'care',
 'cement',
 'child school',
 'childrens',
 'church',
 'city',
 'client',
 'clothes',
 'clothing',
 'college',
 'community',
 'continue',
 'cost',
 'could',
 'cow',
 'currently',
 'customer',
 'dairy',
 'dairy cow',
 'daughter',
 'day',
 'decided',
 'demand',
 'different',
 'domingo',
 'dominican',
 'dream',
 'due',
 'earn',
 'educate',
 'education',
 'enable',
 'enough',
 'entrepreneur',
 'every',
 'every day',
 'expand',
 'expand business',
 'expense',
 'experience',
 'explains',
 'family',
 'farm',
 'farmer',
 'farming',
 'father',
 'fee',
 'feed',
 'first',
 'first loan',
 'five',
 'five child',
 'food',
 'four',
 'four child',
 'friend',
 'fruit',

In [18]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=20,
                                      doc_topic_prior=None,
                                      topic_word_prior=None,
                                      max_iter=200, 
                                      learning_method='batch', 
                                      random_state=123,
                                      n_jobs=2,
                                      verbose=0)
lda_output = lda_model.fit(dtm)

# Log Likelyhood: Higher the better
ll = lda_model.score(dtm)

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
perp = lda_model.perplexity(dtm)

In [19]:
# Theta = document-topic matrix
# Beta = components_ = topic-term matrix
theta = pd.DataFrame(lda_model.transform(dtm))
beta = pd.DataFrame(lda_model.components_)

In [20]:
theta.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.008651,0.008651,0.008651,0.008651,0.008651,0.008651,0.008651,0.008651,0.008651,0.008651,0.008651,0.008651,0.835626,0.008651,0.008651,0.008651,0.008651,0.008651,0.008651,0.008651
1,0.007398,0.007398,0.007398,0.007398,0.007398,0.859442,0.007398,0.007398,0.007398,0.007398,0.007398,0.007398,0.007398,0.007398,0.007398,0.007398,0.007398,0.007398,0.007398,0.007398
2,0.008250,0.008250,0.008250,0.008250,0.008250,0.008250,0.008250,0.008250,0.008250,0.008250,0.008250,0.008250,0.008250,0.008250,0.008250,0.008250,0.629895,0.221606,0.008250,0.008250
3,0.010299,0.010299,0.010299,0.010299,0.010299,0.010299,0.010299,0.010299,0.010299,0.010299,0.010299,0.010299,0.804310,0.010299,0.010299,0.010299,0.010299,0.010299,0.010299,0.010299
4,0.821479,0.009396,0.009396,0.009396,0.009396,0.009396,0.009396,0.009396,0.009396,0.009396,0.009396,0.009396,0.009396,0.009396,0.009396,0.009396,0.009396,0.009396,0.009396,0.009396


In [21]:
beta.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269
0,28.050024,55.885978,48.988058,19.909011,10.189860,2.354766,4.727876,20.474525,0.378533,32.161692,5.142693,8.530279,8.446222,36.124854,33.532371,16.575494,2.146739,11.843563,6.993292,20.687871,0.050000,13.460208,4.655573,24.033933,13.003182,3.493512,6.913491,33.221228,12.130410,6.815911,4.775048,0.575702,8.141661,5.531992,58.906940,1.653023,2.994672,9.873573,1.066259,16.775291,...,2.049603,7.879239,4.402709,0.714309,9.084563,2.146772,6.851696,21.473934,10.672329,15.941316,5.461215,15.564804,23.755323,33.632517,13.934376,7.816909,91.248675,8.684416,1.303330,10.041878,7.751771,30.957027,15.307000,5.007463,3.010610,16.826407,3.902279,0.460238,26.782930,70.164544,76.032001,18.699409,8.562082,12.561041,11.625641,21.284361,27.360169,33.142604,0.050000,13.295494
1,12.353798,0.050000,0.050000,0.050000,0.050000,0.050000,0.307471,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,4.956548,0.050000,0.050000,0.050000,0.050000,0.885992,0.050000,0.050000,0.050000,0.050000,0.366532,0.050000,2.242178,0.050000,0.050000,2.595146,0.050000,0.050000,0.050000,3.795118,0.050000,3.044503,...,0.050000,0.050000,0.050000,0.050000,2.032938,2.599522,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,5.138887,2.601595,1.234308,0.050000,0.050000,0.813433,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.891854,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,2.539648,0.050000
2,12.586755,24.684213,22.409094,7.685108,4.615725,2.389386,4.056614,6.342891,0.479914,20.068030,1.545002,2.530150,14.676912,13.925769,13.595929,6.713187,0.662013,1.554217,1.684779,22.922391,0.240646,7.405079,2.291757,9.284090,7.294385,1.646984,1.230280,19.336675,7.063029,0.771884,1.506407,0.482161,7.268019,1.708439,29.849254,0.218235,0.660072,1.121422,0.229453,16.182310,...,1.213651,5.994107,1.135488,0.498286,4.223973,0.725751,4.186128,10.067901,6.240655,6.713993,0.751210,3.100390,7.253155,18.638213,5.362598,2.550900,41.510594,3.846269,0.177286,3.856132,0.730163,14.818598,8.202997,2.981494,1.330056,6.391474,0.443248,1.505885,13.262739,25.729915,25.377447,5.031498,5.083152,6.820433,5.804337,6.534267,13.890104,16.563073,0.050000,4.510646
3,38.838733,0.824500,0.050000,12.743786,2.420963,8.016108,43.052334,6.001601,13.088882,22.830170,8.212698,7.133148,18.538228,1.895762,0.050000,18.355876,4.206187,16.661741,5.207588,10.344214,2.762269,6.754332,34.827465,1.381208,1.161172,4.596146,26.951284,33.083009,4.830952,7.714367,12.034603,1.817313,2.882189,6.241498,5.794279,16.585758,6.891797,26.715997,51.077750,14.458371,...,4.813492,13.381550,27.389286,5.104114,18.236729,4.907913,1.275671,25.285324,16.342623,32.512320,24.603397,20.679534,0.469682,28.557321,14.681961,3.902933,1.437464,24.487045,16.826310,14.730221,13.274618,11.474574,32.995304,10.075191,2.932143,19.317078,0.050000,4.206603,2.375123,48.822537,4.430356,19.896010,14.307302,26.647323,14.355717,4.389093,14.081605,0.050000,14.013865,21.392063
4,31.457703,8.363259,8.116570,1.794826,0.997785,1.756221,1.628283,3.126880,0.050000,23.366078,1.248667,1.943881,10.633454,0.470899,0.050000,11.718752,0.429315,3.581362,6.039470,3.365297,0.723638,0.672538,1.110626,9.116164,1.226734,11.613765,2.846870,13.214625,6.581154,12.176450,0.986737,0.201347,2.806903,0.773380,4.657792,0.700489,5.998363,11.351242,2.704020,2.844693,...,0.230559,8.563583,21.801339,13.011954,6.029855,0.050000,0.483155,12.855277,5.258332,7.945812,1.321347,8.759704,21.135104,15.872682,5.511497,2.255894,5.897930,2.482183,0.251822,7.238623,4.103181,2.463653,1.931872,2.128915,3.614533,8.309421,6.244404,12.098279,2.345496,4.923134,1.606124,6.808428,2.562854,1.849732,0.209091,3.321678,2.772328,1.7

In [22]:
feature_names = vectorizer.get_feature_names()
weight = theta.sum(axis=0)
support50 = (theta > 0.5).sum(axis=0)
support10 = (theta > 0.1).sum(axis=0)
termss = list()
for topic_id, topic in enumerate(lda_model.components_):
    terms = " ".join([feature_names[i] for i in topic.argsort()[:-6 - 1:-1]])
    termss.append(terms)
topic_summary = pd.DataFrame({'TopicID': range(0, len(termss)), "Support50": support50, "Support10": support10, "Weight": weight, "Terms": termss})

In [27]:
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_columns', 0)
topic_summary

,TopicID,Support50,Support10,Weight,Terms
0,0,692,787,658.712333,group member usd woman group woman school
1,1,71,127,124.272127,fee school fee pay school expand expand business
2,2,381,492,404.407805,cow dairy milk dairy cow group farming
3,3,718,810,668.212187,group life small member dominican hope
4,4,314,388,330.666992,stock primary school primary school able increase
5,5,775,847,709.021666,school requesting requesting loan family buy lack
6,6,136,174,167.686112,expand expand business father family main living
7,7,1,75,70.961151,farming maize land farmer bean farm
8,8,292,382,322.659926,santo domingo santo domingo community entrepre...
9,9,164,230,203.457774,kenya service community yearold applied purchase


In [28]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
% time pyLDAvis.sklearn.prepare(lda_model, dtm, vectorizer, mds="tsne")


CPU times: user 557 ms, sys: 9.82 ms, total: 567 ms
Wall time: 2.17 s


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
5      -10.878068   10.538728       1        1  11.022921
0      -60.884026  -75.591820       2        1   9.989850
19      -2.336119  -53.289074       3        1   9.936971
16     -61.385319  -12.914634       4        1   9.839179
3      -28.737743  126.296997       5        1   9.631401
10       1.066606 -123.552811       6        1   8.471687
14      45.140720   -7.947869       7        1   6.100485
2     -140.795975  -98.530548       8        1   6.056633
12      61.531681  -83.240562       9        1   5.195669
8      -72.563057 -140.396881      10        1   4.939466
4       15.984491   63.578186      11        1   4.822825
9     -160.851624   68.945145      12        1   2.771762
15      85.961830   51.922935      13        1   2.109948
11     -99.020836  109.936348      14        1   1.818199
6     -176.764130   -8.681684      15        1   1.809851
1     -117.189880  -37.420200      16        1   1.513539
17     108.829918  -22.946203      17        1   1.381376
7       47.588097  121.934814      18        1   0.992882
18    -106.467834   30.176889      19        1   0.797678
13     -50.613804   59.328842      20        1   0.797678, topic_info=         Term        Freq       Total Category  logprob  loglift
74    farming  176.000000  176.000000  Default  30.0000  30.0000
139     maize  149.000000  149.000000  Default  29.0000  29.0000
122      land  138.000000  138.000000  Default  28.0000  28.0000
76        fee  149.000000  149.000000  Default  27.0000  27.0000
66     expand  223.000000  223.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
97    grocery    1.049852   89.059722  Topic20  -5.5984   0.3906
191     repay    1.049852  135.134367  Topic20  -5.5984  -0.0264
222    source    1.049852   82.053177  Topic20  -5.5984   0.4725
248  use loan    1.049852  138.706913  Topic20  -5.5984  -0.0525
113   intends    1.049852   93.115723  Topic20  -5.5984   0.3460

[900 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         1  0.119283   able
0         2  0.077534   able
0         3  0.029821   able
0         4  0.128229   able
0         5  0.104372   able
...     ...       ...    ...
269      13  0.024749  young
269      14  0.008250  young
269      17  0.041248  young
269      19  0.008250  young
269      20  0.008250  young

[3754 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 20, 17, 4, 11, 15, 3, 13, 9, 5, 10, 16, 12, 7, 2, 18, 8, 19, 14])